# Theano - classification using logistic regression (base)

### Load all required libraries

In [3]:
import numpy         as np
import itertools     as it
import theano        as th
import theano.tensor as T
th.__version__

'0.8.1'

### Set initial parameters

In [4]:
N_samples      =    10  # training sample size
N_variables    =     3  # number of input variables
training_steps = 10000  # number of training iterations

x_start        = 0
x_stop         = 1
x_numof        = 5

rnd_mul  = 0.001

### Create artificial weighting vector

In [5]:
x_gen    = np.random.randint(1, 4, 1+N_variables)
x_gen

array([1, 1, 3, 3])

These weights are used to create `x_train` as a grid of equally spaced input values.

### Create sample dataset `x_train`

The first column of `x_train` is all ones. 

In [6]:
x_train = np.column_stack([np.ones(x_numof**N_variables),
                           np.array(list(it.product(np.linspace(0,1,5),
                                                    repeat=N_variables)))])

### Create sample vector  `y_train` of target variables 



In [7]:
y_train = np.round(x_train.dot(x_gen)).astype(np.int32)
y_train.shape, y_train

((125,),
 array([1, 2, 2, 3, 4, 2, 2, 3, 4, 5, 2, 3, 4, 5, 6, 3, 4, 5, 6, 6, 4, 5, 6,
        6, 7, 1, 2, 3, 4, 4, 2, 3, 4, 4, 5, 3, 4, 4, 5, 6, 4, 4, 5, 6, 6, 4,
        5, 6, 6, 7, 2, 2, 3, 4, 4, 2, 3, 4, 4, 5, 3, 4, 4, 5, 6, 4, 4, 5, 6,
        7, 4, 5, 6, 7, 8, 2, 2, 3, 4, 5, 2, 3, 4, 5, 6, 3, 4, 5, 6, 6, 4, 5,
        6, 6, 7, 5, 6, 6, 7, 8, 2, 3, 4, 4, 5, 3, 4, 4, 5, 6, 4, 4, 5, 6, 6,
        4, 5, 6, 6, 7, 5, 6, 6, 7, 8], dtype=int32))

NOTES:

- output dimensions = 1 
- input dimensions = N_variables, possibly plus one for the constant term

In [8]:
import logistic_test

In [9]:
sdf = th.shared(np.random.rand(2,3))
sdf.get_value()

array([[ 0.2063372 ,  0.57387022,  0.51234915],
       [ 0.82407766,  0.94629103,  0.49145354]])

In [10]:
index = T.lscalar()  

x = T.matrix ('x')  
y = T.ivector('y')  

classifier = logistic_test.LogisticRegression(x,
                                              n_in=1+N_variables, 
                                              n_out=1)

In [25]:
classifier.W.get_value() # one for each input

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [26]:
classifier.b.get_value() # one for each output

array([ 0.])

In [13]:
y_shared = th.shared(np.asarray(y_train,
                                dtype='int32')
                    )

In [14]:
x_shared = th.shared(np.asarray(x_train,
                                dtype=th.config.floatX)
                    )

In [15]:
unused = T.scalar()
test_model = th.function(
    inputs=[unused],
    outputs=classifier.errors(y),
    on_unused_input='ignore',
    givens={
        x: x_shared,
        y: y_shared
    }
)

In [16]:
test_model(0)

array(1.0)

In [17]:
cost = classifier.negative_log_likelihood(y)

In [18]:
# compute the gradient of cost with respect to theta = (W,b)
g_W = T.grad(cost=cost, wrt=classifier.W)
g_b = T.grad(cost=cost, wrt=classifier.b)

In [19]:
# specify how to update the parameters of the model as a list of
# (variable, update expression) pairs.
learning_rate = 0.1 
updates = [(classifier.W, classifier.W - learning_rate * g_W),
           (classifier.b, classifier.b - learning_rate * g_b)
          ]

In [22]:
train_model = th.function(
    inputs=[x],
    outputs=cost,
    updates=updates
)

In [29]:
train_model(x_train) # ERROR

array(-0.0)

In [30]:
classifier.W.get_value() # one for each input

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [31]:
classifier.b.get_value() # one for each output

array([ 0.])

In [ ]:
classifier.y_pred

In [ ]:
import numpy
predict_model = th.function(
    inputs=[x_shared],
    outputs=classifier.y_pred
)

import numpy
predict_model = th.function(
    inputs=[classifier.input],
    outputs=classifier.y_pred
)

In [ ]:
    ###############
    # TRAIN MODEL #
    ###############
    print('... training the model')
    # early-stopping parameters
    patience = 5000  # look as this many examples regardless
    patience_increase = 2  # wait this much longer when a new best is
                                  # found
    improvement_threshold = 0.995  # a relative improvement of this much is
                                  # considered significant
    validation_frequency = min(n_train_batches, patience // 2)
                                  # go through this many
                                  # minibatche before checking the network
                                  # on the validation set; in this case we
                                  # check every epoch

    best_validation_loss = numpy.inf
    test_score = 0.
    start_time = timeit.default_timer()

    done_looping = False
    epoch = 0
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in range(n_train_batches):

            minibatch_avg_cost = train_model(minibatch_index)
            # iteration number
            iter = (epoch - 1) * n_train_batches + minibatch_index

            if (iter + 1) % validation_frequency == 0:
                # compute zero-one loss on validation set
                validation_losses = [validate_model(i)
                                     for i in range(n_valid_batches)]
                this_validation_loss = numpy.mean(validation_losses)

                print(
                    'epoch %i, minibatch %i/%i, validation error %f %%' %
                    (
                        epoch,
                        minibatch_index + 1,
                        n_train_batches,
                        this_validation_loss * 100.
                    )
                )

                # if we got the best validation score until now
                if this_validation_loss < best_validation_loss:
                    #improve patience if loss improvement is good enough
                    if this_validation_loss < best_validation_loss *  \
                       improvement_threshold:
                        patience = max(patience, iter * patience_increase)

                    best_validation_loss = this_validation_loss
                    # test it on the test set

                    test_losses = [test_model(i)
                                   for i in range(n_test_batches)]
                    test_score = numpy.mean(test_losses)

                    print(
                        (
                            '     epoch %i, minibatch %i/%i, test error of'
                            ' best model %f %%'
                        ) %
                        (
                            epoch,
                            minibatch_index + 1,
                            n_train_batches,
                            test_score * 100.
                        )
                    )

                    # save the best model
                    with open('best_model.pkl', 'wb') as f:
                        pickle.dump(classifier, f)

            if patience <= iter:
                done_looping = True
                break

    end_time = timeit.default_timer()
    print(
        (
            'Optimization complete with best validation score of %f %%,'
            'with test performance %f %%'
        )
        % (best_validation_loss * 100., test_score * 100.)
    )
    print('The code run for %d epochs, with %f epochs/sec' % (
        epoch, 1. * epoch / (end_time - start_time)))
    print(('The code for file ' +
           os.path.split(__file__)[1] +
           ' ran for %.1fs' % ((end_time - start_time))), file=sys.stderr)

